# Build an Index from scratch (if no index is available)
# extract the splice and exons from the genome for the index
!extract_splice_sites.py csi.gene.models.gtf > csi.gene.models.gtf.ss
!extract_exons.py csi.gene.models.gtf  > csi.gene.models.gtf.exon
# Creating a HISAT2 index to csi.gene.models_tran
#!hisat2-build --ss csi.gene.models.ss --exon csi.gene.models.exon csi.chromosome.fa csi.chromosome_tran

# align the RNA-seq reads to the genome
# mapping the reads for each sample to the reference genome
# dta reports alignments tailored for downstream of assemblers (stringtie)
hisat2 -p40 --dta -x /RNA/VALENCIA/csi.chromosome_tran -1  CLV-102-1-GHA_1.fq.qz -2 CLV-102-1-GHA_2.fq.qz -S CLV-102-1-GHA_csi.chromosome.sam


# splits the file name into the base and file type
# uses the base name to create a list
ls -1 *_1.fq.gz | cut -d '_' -f 1 > fastq.list
# create lists to iterate from 
#use the hisat functions and iterate them
for file in $(cat fastq.list); do hisat2 -p 40 --dta -x csi.chromosome_tran \
-1 ${file}_1.fq -2 ${file}_2.fq -S ${file}_.sam; done

# sort and convert the SAM files to BAM
# set path to the folders and convert the files into bam
# rm the sam file to save space
for file in $(cat fastq.list; do samtools sort -@ 40 -o ${file}.bam ${file}_.sam; rm ${file}_.sam; done

In [ ]:
# assemble and quantify expressed genes and transcripts
# -G insert reference annotation -l names prefix name of output transcripts
for file in $(cat fastq.file); do stringtie -p 40 -G csi.gene.models.gtf -o ${file}_csi.chromosome.gtf -l ${file}; done

# merge the transcripts from all samples
stringtie --merge -p 40 -G csi.gene.models.gtf -o stringtie_merged.gtf 

# examine how the transcripts compare w the reference annotation 
gffcompare -r csi.gene.models.gtf -o merged stringtie_merged.gtf


In [ ]:
#  re-estimate transcript abundances using the merged structures and create table counts for ballgown
# -e -G estimates transcripts abundance given by the reference annotations
# -B outputs as a ballgown table files for downstream analysis
for file in $(cat fastq.list); do stringtie -e -B -p 40 -G stringtie_merged.gtf -o ballgown/${file}/${file}_csi.chromosome.gtf ${file}.bam

In [ ]:
# create my own models
# match samples with the same conditions
match(pheno_data[2]
# need to create groups for the function
mod = model.matrix()